In [2]:
import cv2
import numpy as np
from utils.opv import OpvModel
mymodel1=OpvModel("face-detection-adas-0001",device="CPU",fp="FP32")


C:\ProgramData\Anaconda3\envs\BrainAI\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
friends=cv2.imread("friends.jpeg")
friends=cv2.resize(friends,(600,400))
predictions=mymodel1.Predict(friends)

In [ ]:
GENDERS_FOR_OPENVINO=['Female','Male']

In [ ]:
def DrawBoundingBoxes(predictions, image, conf=0.8):
    mymodel2=OpvModel("age-gender-recognition-retail-0013",device="CPU",fp="FP32")
    
    canvas=image.copy()
    predictions_1=predictions[0][0] # numpy array 타입
    confidence=predictions_1[:,2]   # 2번 데이터만
    topresults=predictions_1[(confidence>conf)]  # conf 보다 큰 데이터만
    (h,w)=canvas.shape[:2]
    for detection in topresults:
        box=detection[3:7]*np.array([w,h,w,h])
        (xmin,ymin,xmax,ymax)=box.astype("int")
        img=canvas[ymin:ymax,xmin:xmax].copy()
        img=cv2.resize(img,(62,62))
        blob=cv2.dnn.blobFromImage(img,size=(62,62),ddepth=cv2.CV_8U)
        openvino_net.setInput(blob)
        detections=openvino_net.forwardAndRetrieve(['prob','age_conv3'])
        print(detections)
        gender=GENDERS_FOR_OPENVINO[detections[0][0][0].argmax()]
        age=detections[1][0][0][0][0][0]*100
#         predictions_age=mymodel2.Predict(img)
        print(gender, age)
#         print(predictions_age)
        cv2.rectangle(canvas,(xmin,ymin),(xmax,ymax),(0,0,255),4)
#         cv2.putText(canvas,emo+" "+str(round(max_emo_val*100,1))+"%",(xmin,ymin),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
    cv2.putText(canvas,str(len(topresults))+" face(s) detected",(50,50),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
    
    return canvas

In [ ]:
cv2.imshow("age-gender",DrawBoundingBoxes(predictions,friends))
cv2.waitKey()
cv2.destroyAllWindows()

In [ ]:
import argparse
import glob
import cv2
ap=argparse.ArgumentParser()
ap.add_argument("-x","--xml",required=True,help="path to the IR xml file")
ap.add_argument("-b","--bin",required=True,help="path to the IR bin file")
ap.add_argument("-p","--path",help="path to image files")
ap.add_argument("-t","--target",help="which device should be used for inference")
args=vars(ap.parse_args())


In [ ]:
GENDERS_FOR_OPENVINO=['Female','Male']
openvino_net1=cv2.dnn.readNet('./models/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.xml','./models/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.bin')

In [ ]:
GENDERS_FOR_OPENVINO=['Female','Male']

def detect_age_and_gender_by_openvino(frame):
    faces=face_cascade.detectMultiScale(
    frame,
    scaleFactor=1.2,
    minNeighbors=6,
    minSize=(30,30),
    flags=cv2.CASCADE_SCALE_IMAGE)
    
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w),y+h,(0,255,0),2)
        face=frame[y:y+h,x:x+w]
        blob=cv2.dnn.blobFromImage(face,size=(62,62),ddepth=cv2.CV_8U)
        openvino_net.setInput(blob)
        detections=openvino_net.forwardAndRetrieve(['prob','age_conv3'])
        gender=GENDERS_FOR_OPENVINO[detections[0][0][0].argmax()]
        age=detections[1][0][0][0][0][0]*100
        text="gender={}, age={:.0f}".format(gender,age)
        if gender=='Male':
            color=(255,0,0)
        else:
            color=(0,0,255)
        cv2.putText(frame,text,(10,20),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,1)
        return frame
    
    for file in glob.glob(args["path"]+'/*'):
        filename=str(file)
        image=cv2.imread(filename)
        openvino_image=detect_age_and_gender_by_openvino(image)
        cv2.imshow("OpenVINO",openvino_image)
        cv2.waitKey(0)

# DNN 모듈 사용

In [1]:
import cv2
import numpy as np
age_gender_model=cv2.dnn.readNet('./models/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.xml','./models/age-gender-recognition-retail-0013/FP32/age-gender-recognition-retail-0013.bin')
face_model=cv2.dnn.readNet('./models/face-detection-adas-0001/FP32/face-detection-adas-0001.xml','./models/face-detection-adas-0001/FP32/face-detection-adas-0001.bin')
GENDERS_FOR_OPENVINO=['F','M']


In [2]:
def DrawBoundingBoxes(predictions, image, conf=0.8):    
    canvas=image.copy()
    predictions_1=predictions[0][0] # numpy array 타입
    confidence=predictions_1[:,2]   # 2번 데이터만
    topresults=predictions_1[(confidence>conf)]  # conf 보다 큰 데이터만
    (h,w)=canvas.shape[:2]
    for detection in topresults:
        box=detection[3:7]*np.array([w,h,w,h])
        (xmin,ymin,xmax,ymax)=box.astype("int")
        img=canvas[ymin:ymax,xmin:xmax].copy()
        img=cv2.resize(img,(62,62))
        
        blob=cv2.dnn.blobFromImage(img,size=(62,62),ddepth=cv2.CV_8U)
        age_gender_model.setInput(blob)
        detections=age_gender_model.forwardAndRetrieve(['prob','age_conv3'])
        
        print(detections)
        gender=GENDERS_FOR_OPENVINO[detections[0][0][0].argmax()]
        age=detections[1][0][0][0][0][0]

        cv2.rectangle(canvas,(xmin,ymin),(xmax,ymax),(0,0,255),4)
        if gender=='F':
            cv2.putText(canvas,gender+" "+str(round(age*100,1)),(xmin,ymin),cv2.FONT_HERSHEY_SIMPLEX,0.6,(203,192,255),2)
        else:
            cv2.putText(canvas,gender+" "+str(round(age*100,1)),(xmin,ymin),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)

    cv2.putText(canvas,str(len(topresults))+" face(s) detected",(50,50),cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,0,0),2)
    
    return canvas

In [4]:
img=cv2.imread("images/friends.jpeg")
img=cv2.resize(img,(600,400))

blob=cv2.dnn.blobFromImage(img)
face_model.setInput(blob)
predictions=face_model.forward()

cv2.imshow("age-gender",DrawBoundingBoxes(predictions,img))
cv2.waitKey()
cv2.destroyAllWindows()

[[array([[[[0.9133067 ]],

        [[0.08669331]]]], dtype=float32)], [array([[[[0.21156797]]]], dtype=float32)]]
[[array([[[[0.08485454]],

        [[0.9151455 ]]]], dtype=float32)], [array([[[[0.3399426]]]], dtype=float32)]]
[[array([[[[0.0034712]],

        [[0.9965288]]]], dtype=float32)], [array([[[[0.23634905]]]], dtype=float32)]]
[[array([[[[0.9323675 ]],

        [[0.06763254]]]], dtype=float32)], [array([[[[0.20758618]]]], dtype=float32)]]


### 동영상 인식

In [5]:
camera=cv2.VideoCapture(0)
while True:
    ret,frame=camera.read()
    if not(ret):
        break
    blob=cv2.dnn.blobFromImage(frame)
    face_model.setInput(blob)
    predictions=face_model.forward()
    cv2.imshow('Press Spacebar to Exit',DrawBoundingBoxes(predictions,frame))
    if cv2.waitKey(1)&0xFF == ord(' '):
        break
camera.release()
cv2.destroyAllWindows()

error: OpenCV(4.3.0-openvino-2020.3.0) C:\jenkins\workspace\OpenCV\OpenVINO\2020.3\build\windows\opencv\modules\dnn\src\ie_ngraph.cpp:522: error: (-215:Assertion failed) !isInitialized() in function 'cv::dnn::InfEngineNgraphNet::initPlugin'


# Open CV DNN 모듈
### - 미리 학습된 딥러닝 모델을 이용하여 실행(forward pass, inference)하는 기능
### - 학습은 지원하지 않음
### - OpenCV 3.3 버전부터 기본 기능으로 제공
### - OpenCV 4.3 버전부터 GPU(CUDA) 지원(소스 코드 직접 빌드 필요)
### - 참고 : https://github.com/opencv/opencv/wiki/Deep-Learning-in-OpenCV
### 
## 1) 네트워크 불러오기
### * cv2.dnn.readNet(model, config=None, framework=None) -> retval
### - model : 훈련된 가중치를 저장하고 있는 이진 파일 이름
### - config : 네트워크 구성을 저장하고 있는 텍스트 파일 이름
### - framework : 명시적인 딥러닝 프레임워크 이름
### - retval : cv2.dnn_Net클래스 객체
### 
## 2) 네트워크 입력 블롭(blob) 만들기
### * cv2.dnn.blobFromImage(image, scalefactor=None, size=None, mean=None, 
###             swapRB=None, crop=None, ddepth=None) -> retval
### - image : 입력 영상
### - scalefactor : 입력 영상 픽셀 값에 곱할 값, 기본값은 1. 딥러닝 학습할 때 영상데이터를 픽셀 0~255로 볼 것이냐, 0~1까지 노멀라이즈로 해서 실수값으로 볼것이냐, 0~1이면 1/255를 주고, 1~255이면 1을 주면 된다. 모델파일이 어떤 식으로 학습되었는지 알아야 한다.
### - size : 출력 영상의 크기, 기본값은 (0,0)
### - mean : 입력 영상 각 채널에서 뺄 평균 값, 기본값은 (0,0,0,0). 내가 사용할 데이터셋의 평균
### - swapRB : R과 B 채널을 서로 바꿀 것인지를 결정하는 플래그, 기본값은 False. BGR순서로 되어 있으므로 RGB로 바꿀 필요가 있을 때 결정하는 플래그
### - crop : 크롭 수행 여부, 기본값은 False
### - ddepth : 출력 블롭의 깊이, cv_32F 또는 CV_8U, 기본값은 CV_32F
### - retval : 영상으로부터 구한 블롭 객체
### numpy.ndarray.shape=(N,C,H,W).dtype=numpy.float32
#### DNN객체의 입력 - 4차원형태의 행렬형식의 입력
#### => N : 사진 개수, C : 채널개수 그레이는 2, H,W : 세로와 가로
### 
## 3) 네트워크 입력 설정하기
### * cv2.dnn_Net.setInput(blob, name=None, scalefactor=None, mean=None) -> None
### - blob : 블롭 객체
### - name : 입력 레이어 이름
### - scalefactor : 추가적으로 픽셀 값에 곱할 값
### - mean : 추가적으로 픽셀 값에서 뺄 평균 값
### 
## 4) 네트워크 순방향 실행(추론)
### * cv2.dnn_Net.forward(outputName=None) -> retval
### * cv2.dnn_Net.forward(outputNames=None, outputBlobs=None) -> outputBlobs
### - outputName : 출력 레이어 이름
### - retval : 지정한 레이어의 출력 블롭, 네트워크마다 다르게 결정됨
### - outputNames : 출력 레이어 이름 리스트
### - outputBlobs : 지정한 레이어의 출력 블롭 리스트
### 
### 아래는 dnn 모듈을 이용하여 숫자 인식 예제


In [ ]:
import sys
import numpy as np
import cv2

oldx, oldy = -1,-1

def on_mouse(event, x, y, flags, _):
    global oldx, oldy
    
    if event==cv2.EVENT_LBUTTONDOWN:
        oldx, oldy= x, y
    elif event==cv2.EVENT_LBUTTONUP:
        oldx, oldy= -1, -1
    elif event==cv2.EVENT_MOUSEMOVE:
        if flags & cv2.EVENT_FLAG_LBUTTON:
            cv2.line(img,(oldx, oldy),(x,y),(255,255,255),40,cv2.LINE_AA)
            oldx,oldy=x,y
            cv2.imshow('img',img)

def norm_digit(img):
    m=cv2.moments(img)
    cx=m['m10']/m['m00']
    cy=m['m01']/m['m00']
    h,w=img.shape[:2]
    aff=np.array([[1,0,w/2-cx],[0,1,h/2-cy]], dtype=np.float32)
    return dst
    
net=cv2.dnn.readNet('mnist_cnn.pb')  # mnist_cnn.pb 파일로부터 네트워크 생성

if net.empty():
    print('Network load failed')
    sys.exit()
    
img=np.zeros((400,400), np.uint8)
cv2.imshow('img',img)
cv2.setMouseCallback('img',on_mouse)

while True :
    c=cv2.waitKey()
    if c==27:
        break
    elif c==ord(' '):
        blob=cv2.dnn.blobFromImage(norm_digit(img), 1/255., (28,28)) # blob : shape=(1,1,28,28). dtype=float32
        net.setInput(blob)                               
        prob=net.forward()                               # prob : shape=(1,10). dtype=float32
        
        _, maxVal, _, maxLoc = cv2.minMaxLoc(prob)       # prob 행렬에서 최대값 위치가 인식된 숫자를 나타냄
        digit=maxLoc[0]                                 
        
        print(f'{digit} ({maxVal*100:4.2f}%)')
        img.fill(0)
        cv2.imshow('img',img)
cv2.destroyAllWindows